# Run Streamlit and Bedrock on SageMaker notebook instance

This notebook was built using the conda PyTorch 3.10 kernel on an `ml.t3.medium` instance.

### Step 1. Install Streamlit

In [ ]:
!pip install streamlit

### Step 2. Write a minimal Streamlit app with Python

In [ ]:
%%writefile language_app.py

import streamlit as st

def add_to_session_history(role, content):
    st.session_state.messages.append({"role": role, "content": content})
    

# this adds a title to the application
st.title('My first language application')

# create a local list to hold the mesage history
st.session_state.messages = []


# this prints all the messages in the history
for msg in st.session_state.messages:
    with st.chat_message(msg['role']):
        st.markdown(msg['content'])

# this creates a template on the screen and a variable in the script
prompt = st.chat_input('This is where you put your questions')

if prompt:
    
    st.chat_message('user').markdown(prompt)
    
    add_to_session_history('user', prompt)
    
    res = f'I heard you say: {prompt}'
    
    with st.chat_message('assistant'):
        st.markdown(res)
        
    add_to_session_history('assistant', res)

    

Remember, to view this in your SageMaker notebook instance you'll need to follow a few steps.
1. Copy the url you see above for your notebook instance.
2. After the `sagemaker.aws/` part of the url, add `proxy/8501/`. This port number, here `8501`, will change when you run additional Streamlit apps. Each time you invoke `streamlit run <>`, Streamlit will tell you which port to use in the output. Make sure you use that number when you update the url.
3. Open a new tab in your web browswer, preferably one with a light background, and paste this in! You'll need to be logged in to your AWS account in that web browser to access this.
4. Every time you modify this script, just remember to stop your Kernel and run the command again below.

In [ ]:
!streamlit run language_app.py

Before you move onto step 3, stop your Jupyter kernel. You can open up a terminal and run the commandss `pip install streamlit && streamlit run language_app.py` to keep the front end active if you like.

### Step 3. Test your Bedrock connection
Before continuing, make sure the IAM role you are using has access to Bedrock. Go do this now by updating the SageMaker execution role in the IAM console within AWS now if you don't already have this. Once you've added the relevant permissions, come back here and test the Bedrock connection.

In [ ]:
import boto3
import json

bedrock_client = boto3.client('bedrock-runtime')

Bedrock uses a `Messages` API, meaning it is explicitly looking for a list of messages in order to respond. This is helpful because it simplifies the development for chat!

In [ ]:
messages=[{ "role":'user', 
           "content":[{'type':'text',
                       'text': "What does it mean to train a language model?"}]}]


In [ ]:
body = json.dumps(
        {
            "max_tokens": 100,
            "messages": messages,
            "temperature": 0.5,
            "top_p": 1,
            # you'll need to pass an anthropic version here
            "anthropic_version": "bedrock-2023-05-31",    

        }  
    )  

In [ ]:
response = bedrock_client.invoke_model(body=body, 
                                       # also pass a model ID here
                                       modelId="anthropic.claude-3-sonnet-20240229-v1:0")

In [ ]:
response_body = json.loads(response.get('body').read())

In [ ]:
response_body['content'][0]['text']

### Step 4. Connect Bedrock to Streamlit
Now that you have a connection to Bedrock working, let's integrate this into our minimal Streamlit application. This will be a few key steps.
1. Initialize the chat history with a system message, giving the language model some high level context. 
2. Capture incoming messages from the customer.
3. Format the messages to invoke Bedrock
4. Send the messages and receive the response
5. Send the response to the customer

In [ ]:
%%writefile bedrock_language_app.py

import streamlit as st
import json
import boto3

def add_to_session_history(role, content):
    st.session_state.messages.append({"role": role, "content": content})

def initialize_msg_history():
    # create a local list to hold the mesage history
    st.session_state.messages = []

def send_to_bedrock(bedrock_client):
    
    bedrock_messages = st.session_state.messages

    body = json.dumps({ "max_tokens": 512,
                    "messages": bedrock_messages,
                    "temperature": 0.5,
                    "top_p": 1,
                    "anthropic_version": "bedrock-2023-05-31"})  
    
    response = bedrock_client.invoke_model(body=body, modelId="anthropic.claude-3-sonnet-20240229-v1:0")
    
    response_body = json.loads(response.get('body').read())
    
    return response_body['content'][0]['text']
    
                                
def handle_prompt(prompt, bedrock_client):
    
    st.chat_message('user').markdown(prompt)

    add_to_session_history('user', prompt)

    # this looks in the local session state history managed by Streamlit 
    res = send_to_bedrock(bedrock_client)

    with st.chat_message('assistant'):
        st.markdown(res)

    add_to_session_history('assistant', res)


if __name__ == '__main__':

    # this adds a title to the application
    st.title('My Bedrock language application')
    
    initialize_msg_history()

    # do this once per session
    bedrock_client = boto3.client('bedrock-runtime')

    # this creates a template on the screen and a variable in the script
    prompt = st.chat_input('This is where you put your questions')

    # is triggered when the user hits enter in the on-screen widget
    if prompt:

        handle_prompt(prompt, bedrock_client)
        

In [1]:
!streamlit run bedrock_language_app.py




  You can now view your Streamlit app in your browser.

  Network URL: http://172.16.64.209:8502
  External URL: http://34.203.37.48:8502

^C
  Stopping...
